# Use subqueries to refine data

## Scenario

I work for an organization that is responsible for the safety, efficiency, and maintenance of transportation systems in my city. I have been asked to gather information around the use of Citi Bikes in New York City. This information will be used to convince the mayor and other city officials to invest in a bike sharing and rental system to help push the city toward its environmental and sustainability goals.

For this purpose, I will use 3 SQL queries to gather information:

- about the average trip duration by station,
- to compare trip duration by station, and
- to determine the five stations with the longest mean trip durations.

## Dataset

I will obtain this information using the BigQuery public dataset `new_york_citibike` with the full path `bigquery-public-data.new_york_citibike`. This dataset has the following tables:

- citibike_stations
- citibike_trips

## Query: Average trip duration by station

To find the average trip duration by station, I execute the following query on the `citibike_trips` table containing a subquery in a FROM statement:

In [ ]:
/* Outer query to obtain the station id and average trip
   duration of trips started from each */
SELECT
    avg_trip_duration.start_station_id,
    avg_trip_duration.avg_duration

FROM
	-- Subquery to calculate average trip duration
    (
		SELECT
            start_station_id,
            ROUND(AVG(tripduration), 2) as avg_duration
	    FROM
            bigquery-public-data.new_york_citibike.citibike_trips
        GROUP BY
            start_station_id
	) AS
    avg_trip_duration -- Subquery alias

ORDER BY avg_duration DESC;

The query returns a list of every station id with the average trip duration of trips that started from each station. Below is a preview of the output:

![Average trip duration by station](c05m03-query-avg-trip-duration.png 'Average trip duration by station')

## Query: Compare trip duration by station

To compare the trip duration to the overall average trip duration for each station, I use the `citibike_trips` table to execute the following query that contains two subqueries in the a SELECT statement:

In [ ]:
/* Outer query to obtain start time, station id, trip duration, average
   trip duration for specific station and how much trip duration differ
   from specific station's overall average trip duration */
SELECT
    -- Clean date formatting in 2 separated columns
    CAST(starttime AS STRING FORMAT "yyyy-mm-dd") AS start_date,
    CAST(starttime AS STRING FORMAT "hh:mm:ss") AS start_time,
    start_station_id,
    ROUND((tripduration/60), 2) AS trip_minutes, -- Convert seconds to minutes
    
    -- Subquery to calculate average trip duration for each station
    (
        SELECT
            ROUND(AVG(tripduration/60),2)
        FROM
            bigquery-public-data.new_york_citibike.citibike_trips
        WHERE
            start_station_id = trips.start_station_id
    )
    AS avg_duration_station,
    
    /* Subquery to calculate difference between trip duration
       and specific station's overall average trip duration */
    ROUND((tripduration/60) -
        (
            SELECT
                AVG(tripduration/60)
            FROM
                bigquery-public-data.new_york_citibike.citibike_trips
            WHERE
                start_station_id = trips.start_station_id),
        2)
        AS difference_from_avg

FROM
    bigquery-public-data.new_york_citibike.citibike_trips AS trips

WHERE
    start_station_id IS NOT NULL

ORDER BY
    difference_from_avg DESC

LIMIT 25;

Query output returns a table with the start date, start time, start station id, the trip duration, the average trip duration for the start station and finally the difference between each trip and the overall average for that station:

![Compare trip duration to station average](c05m03-query-compare-station-avg.png 'Compare trip duration to station average')